In [1]:
# pip install imbalanced-learn

In [2]:
import numpy as np
%pip install pandas
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
data = pd.read_csv("Datasets\\Crop and fertilizer dataset.csv")
# data = data.drop(columns = ["Crop Type","Soil Type"], axis = 1)
data.head()

,District_Name,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer,Link
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo


In [4]:
new_column_names = {
    'Nitrogen': 'N',
    'Potassium': 'K',
    'Phosphorus': 'P',
    'Temperature': 'temperature',
    'Fertilizer': 'fertilizer'
}

data.rename(columns = new_column_names, inplace = True)

data = data.reindex(columns = ['temperature', 'N', 'K', 'P', 'fertilizer'])
print(data.head())
print(data.value_counts('fertilizer'))

   temperature   N    K   P fertilizer
0           20  75  100  50       Urea
1           20  80  100  50       Urea
2           20  85  100  50       Urea
3           20  90  100  50       Urea
4           20  95  100  50       Urea
fertilizer
Urea                      1364
DAP                        667
MOP                        571
19:19:19 NPK               480
SSP                        417
Magnesium Sulphate         215
10:26:26 NPK               156
50:26:26 NPK               124
Chilated Micronutrient     108
12:32:16 NPK               106
Ferrous Sulphate            68
13:32:26 NPK                66
10:10:10 NPK                50
Ammonium Sulphate           50
Hydrated Lime               25
White Potash                19
20:20:20 NPK                15
Sulphur                      6
18:46:00 NPK                 6
Name: count, dtype: int64


In [5]:
data.columns

Index(['temperature', 'N', 'K', 'P', 'fertilizer'], dtype='object')

In [6]:
features = ['N', 'K', 'P', 'temperature']

X = data[features]
X.head()

,N,K,P,temperature
0,75,100,50,20
1,80,100,50,20
2,85,100,50,20
3,90,100,50,20
4,95,100,50,20


In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.5363069 ,  1.02008727, -0.26234803, -1.00313047],
       [-0.40492307,  1.02008727, -0.26234803, -1.00313047],
       [-0.27353923,  1.02008727, -0.26234803, -1.00313047],
       ...,
       [ 1.17168299,  2.42112012,  1.55032358,  0.6927406 ],
       [ 1.30306682,  2.42112012,  1.55032358,  0.6927406 ],
       [ 1.43445066,  2.42112012,  1.55032358,  0.6927406 ]])

In [8]:
pca = PCA(n_components = len(features))
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[ 0.09465764, -0.8541516 ,  1.27884329, -0.17124596],
       [ 0.17250667, -0.85588121,  1.22935262, -0.07771028],
       [ 0.25035569, -0.85761082,  1.17986196,  0.01582541],
       ...,
       [ 2.93171412,  0.81906803,  0.86550109, -0.32116071],
       [ 3.00956314,  0.81733842,  0.81601042, -0.22762503],
       [ 3.08741217,  0.81560882,  0.76651976, -0.13408934]])

In [9]:
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio

array([0.5618533 , 0.25259292, 0.11331181, 0.07224197])

In [10]:
eigenvectors = pca.components_
eigenvectors

array([[ 0.59253122,  0.54861785,  0.58980407, -0.00750857],
       [-0.01316454,  0.11035481, -0.07680937,  0.99083239],
       [-0.37668761,  0.82860092, -0.39393743, -0.12782899],
       [ 0.71192689, -0.01614875, -0.70074588, -0.04306436]])

In [11]:
weights = eigenvectors[0] * explained_variance_ratio[0]
for i in range(1, len(features)):
  weights += eigenvectors[i] * explained_variance_ratio[i]
weights

array([0.33833819, 0.42884125, 0.21672084, 0.22846294])

In [12]:
weights /= weights.sum()
weights

array([0.27907329, 0.35372341, 0.178759  , 0.1884443 ])

In [13]:
def calculate_sqi(X_scaled, weights):
  return (X_scaled * weights).sum(axis = 1)

In [14]:
sqi= calculate_sqi(X_scaled, weights)
sqi

array([-0.02477149,  0.01189423,  0.04855995, ...,  1.59106959,
        1.62773531,  1.66440103])

In [15]:
minmax = MinMaxScaler()
sqi_scaled = minmax.fit_transform(sqi.reshape(-1, 1))
sqi_scaled

array([[0.47271634],
       [0.48343409],
       [0.49415185],
       ...,
       [0.94504289],
       [0.95576065],
       [0.9664784 ]])

In [16]:
data['SQI'] = sqi_scaled
data.head()

,temperature,N,K,P,fertilizer,SQI
0,20,75,100,50,Urea,0.472716
1,20,80,100,50,Urea,0.483434
2,20,85,100,50,Urea,0.494152
3,20,90,100,50,Urea,0.504870
4,20,95,100,50,Urea,0.515587


In [17]:
print(min(data['SQI']))
print(max(data['SQI']))

0.0
0.9999999999999999


In [18]:
fert_data = pd.read_csv('Datasets\\Crop and fertilizer dataset.csv')
fert_data = fert_data.drop(columns = ['District_Name', 'Soil_color', 'pH', 'Rainfall', 'Crop', 'Link'])
fert_data.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Fertilizer
0,75,50,100,20,Urea
1,80,50,100,20,Urea
2,85,50,100,20,Urea
3,90,50,100,20,Urea
4,95,50,100,20,Urea


In [19]:
new_column_names = {
    'Nitrogen': 'N',
    'Potassium': 'K',
    'Phosphorus': 'P',
    'Temperature': 'temperature',
    'Fertilizer': 'fertilizer'
}

fert_data.rename(columns = new_column_names, inplace = True)

fert_data = fert_data.reindex(columns = ['temperature', 'N', 'K', 'P', 'fertilizer'])
print(fert_data.head())
print(fert_data.value_counts('fertilizer'))

   temperature   N    K   P fertilizer
0           20  75  100  50       Urea
1           20  80  100  50       Urea
2           20  85  100  50       Urea
3           20  90  100  50       Urea
4           20  95  100  50       Urea
fertilizer
Urea                      1364
DAP                        667
MOP                        571
19:19:19 NPK               480
SSP                        417
Magnesium Sulphate         215
10:26:26 NPK               156
50:26:26 NPK               124
Chilated Micronutrient     108
12:32:16 NPK               106
Ferrous Sulphate            68
13:32:26 NPK                66
10:10:10 NPK                50
Ammonium Sulphate           50
Hydrated Lime               25
White Potash                19
20:20:20 NPK                15
Sulphur                      6
18:46:00 NPK                 6
Name: count, dtype: int64


In [20]:
from imblearn.over_sampling import SMOTE
# from google.colab import files

X = fert_data.drop(columns = ['fertilizer'], axis = 1)
y = fert_data['fertilizer']

smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X, y)

df_resampled = pd.concat([pd.DataFrame(X_resampled, columns = X.columns), pd.Series(y_resampled, name = 'fertilizer')], axis = 1)
print(df_resampled.head())
print(df_resampled.value_counts('fertilizer'))
# df_resampled.to_csv('New Crop Dataset.csv', index = False)
# files.download('New Crop Dataset.csv')

   temperature   N    K   P fertilizer
0           20  75  100  50       Urea
1           20  80  100  50       Urea
2           20  85  100  50       Urea
3           20  90  100  50       Urea
4           20  95  100  50       Urea
fertilizer
10:10:10 NPK              1364
DAP                       1364
Urea                      1364
Sulphur                   1364
SSP                       1364
Magnesium Sulphate        1364
MOP                       1364
Hydrated Lime             1364
Ferrous Sulphate          1364
Chilated Micronutrient    1364
10:26:26 NPK              1364
Ammonium Sulphate         1364
50:26:26 NPK              1364
20:20:20 NPK              1364
19:19:19 NPK              1364
18:46:00 NPK              1364
13:32:26 NPK              1364
12:32:16 NPK              1364
White Potash              1364
Name: count, dtype: int64


In [21]:
X = df_resampled.drop("fertilizer", axis = 1)
Y = df_resampled["fertilizer"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [22]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators = 100, random_state = 42)
model.fit(X_train, Y_train)

ExtraTreesClassifier(random_state=42)

In [23]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(Y_test, y_pred, zero_division = 0))

                        precision    recall  f1-score   support

          10:10:10 NPK       1.00      0.99      0.99       260
          10:26:26 NPK       0.88      0.91      0.90       283
          12:32:16 NPK       0.93      0.98      0.96       284
          13:32:26 NPK       0.98      1.00      0.99       282
          18:46:00 NPK       0.94      0.91      0.93       281
          19:19:19 NPK       0.83      0.84      0.84       271
          20:20:20 NPK       1.00      0.99      1.00       287
          50:26:26 NPK       0.87      0.91      0.89       259
     Ammonium Sulphate       0.99      1.00      0.99       274
Chilated Micronutrient       0.89      0.96      0.93       263
                   DAP       0.80      0.77      0.78       245
      Ferrous Sulphate       0.98      0.95      0.96       265
         Hydrated Lime       1.00      1.00      1.00       279
                   MOP       0.81      0.78      0.79       279
    Magnesium Sulphate       0.92      

In [24]:
data['Fert_Predict'] = model.predict(fert_data.drop(columns = ["fertilizer"]))
data

,temperature,N,K,P,fertilizer,SQI,Fert_Predict
0,20,75,100,50,Urea,0.472716,Urea
1,20,80,100,50,Urea,0.483434,Urea
2,20,85,100,50,Urea,0.494152,Urea
3,20,90,100,50,Urea,0.504870,Urea
4,20,95,100,50,Urea,0.515587,Urea
...,...,...,...,...,...,...,...
4508,30,130,150,80,MOP,0.923607,MOP
4509,30,135,150,80,MOP,0.934325,MOP
4510,30,140,150,80,MOP,0.945043,MOP
4511,30,145,150,80,MOP,0.955761,MOP


In [25]:
import pickle

In [26]:
pickle_file = 'fertilizerPrediction.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)